In [ ]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from pprint import pprint

In [ ]:
class feature_weight :
    def __init__(self) :
        iris = datasets.load_iris()
        self.data, self.target = pd.DataFrame(iris.data, columns=iris.feature_names), iris.target

    def global_accuracy(self) :
        """
        Calculate Global Knn accuracy
        """
        global_X_train, global_X_val, global_y_train, global_y_val = train_test_split(self.data, self.target)
    
        knn = KNeighborsClassifier(n_neighbors=5)
        knn.fit(global_X_train, global_y_train)
        self.original_acc = accuracy_score(knn.predict(global_X_val), global_y_val)
        
        
    def feature_iteration(self) :
        """
        Calculate each accuarcy that remove one column
        """
        self.global_accuracy()
        self.weight = []
        
        for col in self.data :
            #remove one column
            column = list(self.data.columns)
            column.remove(col)
            
            #make new dataset that remove one column
            adjusted_data = self.data[column]
            X_train, X_val, y_train, y_val = train_test_split(adjusted_data, self.target)
            
            #learning using Knn
            knn = KNeighborsClassifier(n_neighbors=5)
            knn.fit(X_train, y_train)
            self.weight.append(accuracy_score(knn.predict(X_val), y_val))
            
    def get_weight(self) :
        """
        Calculate weight using paper
        """
        self.feature_iteration()
        for idx in range(len(self.weight)) :
            self.weight[idx] = 1-(self.weight[idx]-self.original_acc)
            
        sum_of_disc = sum(self.weight)
        for idx in range(len(self.weight)) :
            self.weight[idx] = self.weight[idx]/sum_of_disc
            
        return self.weight

In [ ]:
iris = datasets.load_iris()
data, target = pd.DataFrame(iris.data, columns=iris.feature_names), iris.target

# Weighted Euclidean distance

In [ ]:
def weigthed_l2(weight, row1, row2) :
  sub = row1-row2
  return np.sqrt((weight*sub*sub).sum())

In [ ]:
def normal_l2(row1, row2) :
  sub = row1-row2
  return np.sqrt((sub*sub).sum())

In [ ]:
normal_l2(data.iloc[0, :], data.iloc[1, :])

0.5385164807134502

In [ ]:
weight = feature_weight().get_weight()
weigthed_l2(weight, data.iloc[0, :], data.iloc[1, :])

0.2705549851693735

**계산기로 계산해본 결과 위 결과와 동일** 

# 각 Point별로 weighted euclidean distance 구하기

In [ ]:
dist_matrix = np.zeros((len(target), len(target)))

mat_data = np.array(data)
for idx_0, value_0 in enumerate(mat_data) :
  for idx_1, value_1 in enumerate(mat_data) :
    dist_matrix[idx_0, idx_1] = weigthed_l2(weight, value_0, value_1)

In [87]:
# for test
k_list = [1,2,3,4,5]

idx_list = []
value_list = []
for idx, value in enumerate(dist_matrix[101]) :
  idx_list.append(idx)
  value_list.append(value)

sort_df = pd.DataFrame((idx_list, value_list), index=['index', 'value']).T
sort_df = sort_df.sort_values(by=['value'])
sorted_index_list = list(sort_df.iloc[1:k_list[4]+1, :].index)

print(target[101]==[target[i] for i in sorted_index_list])
print((target[101]==[target[i] for i in sorted_index_list]).mean())

[ True  True  True  True False]
0.8


In [143]:
k_list = [1,2,3,4,5]
local_k_df = []
local_k_list = []

for k in k_list :
  acc_list = []

  for i , row in enumerate(dist_matrix) :
    idx_list = []
    value_list = []
    
    for idx, value in enumerate(row) :
      idx_list.append(idx)
      value_list.append(value)

    sort_df = pd.DataFrame((idx_list, value_list), index=['index', 'value']).T
    sort_df = sort_df.sort_values(by=['value'])
    sort_df = list(sort_df.iloc[1:k+1, :].index)

    acc_list.append((target[i]== [target[j] for j in sort_df]).mean())
  local_k_df.append(acc_list)

In [150]:
local_k_df = np.array(local_k_df).T
pprint(local_k_df)

array([[1.        , 1.        , 1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.